In [ ]:
!pip install fastapi
!pip install colabcode
!pip install fastapi

In [ ]:
!pip install -qU ddsp==1.6.5

In [ ]:
# !pip install librosa

In [ ]:
!pip install pydub

In [ ]:
!pip install pyrebase

In [ ]:
!pip install python-multipart

In [ ]:
## !pip install -U "tensorflow==2.0"

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import copy
import os
import time

import crepe
import ddsp
import ddsp.training
from ddsp.colab.colab_utils import (
    auto_tune, get_tuning_factor, download, 
    play, record, specplot, upload, 
    DEFAULT_SAMPLE_RATE)
from ddsp.training.postprocessing import (
    detect_notes, fit_quantile_transform
)
import gin
from google.colab import files
import librosa
import matplotlib.pyplot as plt
import numpy as np
import pickle
import tensorflow.compat.v2 as tf
import tensorflow_probability as tfp
import tensorflow_datasets as tfds

# Helper Functions
sample_rate = DEFAULT_SAMPLE_RATE  # 16000

print('Done!')

In [ ]:
from IPython.display import Audio
from colabcode import ColabCode
cc = ColabCode(port=8000, code=False)

In [ ]:
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list && sudo apt update && sudo apt install ngrok   

In [ ]:
!ngrok authtoken #enter ngrok authtoken here

In [ ]:
from google.colab import files
import os
from fastapi import FastAPI, Request, Form
import shutil
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
import pyrebase
import librosa
import numpy as np
import base64
import io
import pickle
import soundfile as sf

import ddsp.training
from google.colab import files
from google.colab import output
from IPython import display
import note_seq
import numpy as np
from scipy.io import wavfile





app = FastAPI()

config = {
    #add firebase config details here 
}

firebase = pyrebase.initialize_app(config)    
storage = firebase.storage()

origins = [
    "http://localhost:3000/",
    "http://localhost:3000"
]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=False,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.get("/")
async def read_root():
  print("Came")
  return "Hello World"

@app.post("/convert")
def upload_file(songfileName: str = Form(...), instrumentType: str = Form(...), uid: str = Form(...)):

  # with open(filename, "wb") as buffer:
  #   shutil.copyfileobj(audio_file.file, buffer)
  print(instrumentType)

  try:
    storage.child(uid + "/vocals/" + songfileName).download(songfileName)
    
    # filenames, audios = upload()
    # audio = audios[0]
    # if len(audio.shape) == 1:
    #   audio = audio[np.newaxis, :]
    # print('\nExtracting audio features...')

    DEFAULT_SAMPLE_RATE = ddsp.spectral_ops.CREPE_SAMPLE_RATE
    def audio_bytes_to_np(wav_data,
                      sample_rate=DEFAULT_SAMPLE_RATE,
                      normalize_db=0.1,
                      mono=True):
      return note_seq.audio_io.wav_data_to_samples_pydub(wav_data=wav_data, sample_rate=sample_rate, normalize_db=normalize_db,num_channels=1 if mono else None)
      
    with open(songfileName, 'rb') as fd:
      data = fd.read()

    file_audio = audio_bytes_to_np(data,sample_rate=sample_rate, normalize_db= None)
    #print(file_audio)


    # #Setup the session.
    ddsp.spectral_ops.reset_crepe()


    # Compute features.
    start_time = time.time()
    print(start_time)
    #print("came here")
    #print(ddsp.training.metrics.compute_audio_features(file_audio))
    audio_features = ddsp.training.metrics.compute_audio_features(file_audio)
    #print(audio_features)
    audio_features['loudness_db'] = audio_features['loudness_db'].astype(np.float32)
    audio_features_mod = None
    print('Audio features took %.1f seconds' % (time.time() - start_time))

    TRIM = -15
    print("Hello")

    model =  instrumentType
    MODEL = model
    def find_model_dir(dir_name):
      # Iterate through directories until model directory is found
      for root, dirs, filenames in os.walk(dir_name):
        for filename in filenames:
          if filename.endswith(".gin") and not filename.startswith("."):
            model_dir = root
            break
      return model_dir 

    if model in ('Violin', 'Flute', 'Flute2', 'Trumpet', 'Tenor_Saxophone'):
      # Pretrained models.
      PRETRAINED_DIR = '/content/pretrained'
      # Copy over from gs:// for faster loading.
      !rm -r $PRETRAINED_DIR &> /dev/null
      !mkdir $PRETRAINED_DIR &> /dev/null
      GCS_CKPT_DIR = 'gs://ddsp/models/timbre_transfer_colab/2021-07-08'
      model_dir = os.path.join(GCS_CKPT_DIR, 'solo_%s_ckpt' % model.lower())
      
      !gsutil cp $model_dir/* $PRETRAINED_DIR &> /dev/null
      model_dir = PRETRAINED_DIR
      gin_file = os.path.join(model_dir, 'operative_config-0.gin')

    else:
      # User models.
      UPLOAD_DIR = '/content/uploaded'
      !mkdir $UPLOAD_DIR
      uploaded_files = files.upload()  # check

      for fnames in uploaded_files.keys():
        print("Unzipping... {}".format(fnames))
        !unzip -o "/content/$fnames" -d $UPLOAD_DIR &> /dev/null
      model_dir = find_model_dir(UPLOAD_DIR)
      gin_file = os.path.join(model_dir, 'operative_config-0.gin')


    # Load the dataset statistics.
    DATASET_STATS = None
    dataset_stats_file = os.path.join(model_dir, 'dataset_statistics.pkl')
    print(f'Loading dataset statistics from {dataset_stats_file}')
    try:
      if tf.io.gfile.exists(dataset_stats_file):
        with tf.io.gfile.GFile(dataset_stats_file, 'rb') as f:
          DATASET_STATS = pickle.load(f)
    except Exception as err:
      print('Loading dataset statistics from pickle failed: {}.'.format(err))

    
    # Parse gin config,
    with gin.unlock_config():
      gin.parse_config_file(gin_file, skip_unknown=True)

    # Assumes only one checkpoint in the folder, 'ckpt-[iter]`.
    ckpt_files = [f for f in tf.io.gfile.listdir(model_dir) if 'ckpt' in f]
    ckpt_name = ckpt_files[0].split('.')[0]
    ckpt = os.path.join(model_dir, ckpt_name)

    print("ckpt files done")
    
    # Ensure dimensions and sampling rates are equal
    time_steps_train = gin.query_parameter('F0LoudnessPreprocessor.time_steps')
    n_samples_train = gin.query_parameter('Harmonic.n_samples')
    hop_size = 1000

    time_steps = 1000
    n_samples = time_steps * hop_size

    # print("===Trained model===")
    # print("Time Steps", time_steps_train)
    # print("Samples", n_samples_train)
    # print("Hop Size", hop_size)
    # print("\n===Resynthesis===")
    # print("Time Steps", time_steps)
    # print("Samples", n_samples)
    # print('')

    gin_params = [
        'Harmonic.n_samples = {}'.format(n_samples),
        'FilteredNoise.n_samples = {}'.format(n_samples),
        'F0LoudnessPreprocessor.time_steps = {}'.format(time_steps),
        'oscillator_bank.use_angular_cumsum = True',  # Avoids cumsum accumulation errors.
    ]

    with gin.unlock_config():
      gin.parse_config(gin_params)


    # print("Trim all input vectors to correct lengths") 
    # for key in ['f0_hz', 'f0_confidence', 'loudness_db']:
    #   audio_features[key] = audio_features[key][:time_steps]
    # audio_features['audio'] = audio_features['audio'][:, :n_samples]

    # print("below input vectors")
    # Set up the model just to predict audio given new conditioning
    model = ddsp.training.models.Autoencoder()
    model.restore(ckpt)

    # Build model by running a batch through it.
    start_time = time.time()
    _ = model(audio_features, training=False)
    print('Restoring model took %.1f seconds' % (time.time() - start_time))

    threshold = 1
    ADJUST = True
    quiet = 20
    autotune = 0
    pitch_shift =  0
    loudness_shift = 0

    audio_features_mod = {k: v.copy() for k, v in audio_features.items()}


    ## Helper functions.
    def shift_ld(audio_features, ld_shift=0.0):
      """Shift loudness by a number of ocatves."""
      audio_features['loudness_db'] += ld_shift
      return audio_features


    def shift_f0(audio_features, pitch_shift=0.0):
      """Shift f0 by a number of ocatves."""
      audio_features['f0_hz'] *= 2.0 ** (pitch_shift)
      audio_features['f0_hz'] = np.clip(audio_features['f0_hz'], 
                                        0.0, 
                                        librosa.midi_to_hz(110.0))
      return audio_features


    mask_on = None

    if ADJUST and DATASET_STATS is not None:
      # Detect sections that are "on".
      mask_on, note_on_value = detect_notes(audio_features['loudness_db'],
                                            audio_features['f0_confidence'],
                                            threshold)

      if np.any(mask_on):
        # Shift the pitch register.
        target_mean_pitch = DATASET_STATS['mean_pitch']
        pitch = ddsp.core.hz_to_midi(audio_features['f0_hz'])
        mean_pitch = np.mean(pitch[mask_on])
        p_diff = target_mean_pitch - mean_pitch
        p_diff_octave = p_diff / 12.0
        round_fn = np.floor if p_diff_octave > 1.5 else np.ceil
        p_diff_octave = round_fn(p_diff_octave)
        audio_features_mod = shift_f0(audio_features_mod, p_diff_octave)


        # Quantile shift the note_on parts.
        _, loudness_norm = fit_quantile_transform(
            audio_features['loudness_db'],
            mask_on,
            inv_quantile=DATASET_STATS['quantile_transform'])

        # Turn down the note_off parts.
        mask_off = np.logical_not(mask_on)
        loudness_norm[mask_off] -=  quiet * (1.0 - note_on_value[mask_off][:, np.newaxis])
        loudness_norm = np.reshape(loudness_norm, audio_features['loudness_db'].shape)
        
        audio_features_mod['loudness_db'] = loudness_norm 

        # Auto-tune.
        if autotune:
          f0_midi = np.array(ddsp.core.hz_to_midi(audio_features_mod['f0_hz']))
          tuning_factor = get_tuning_factor(f0_midi, audio_features_mod['f0_confidence'], mask_on)
          f0_midi_at = auto_tune(f0_midi, tuning_factor, mask_on, amount=autotune)
          audio_features_mod['f0_hz'] = ddsp.core.midi_to_hz(f0_midi_at)

      else:
        print('\nSkipping auto-adjust (no notes detected or ADJUST box empty).')

    else:
      print('\nSkipping auto-adujst (box not checked or no dataset statistics found).')

    # Manual Shifts.
    audio_features_mod = shift_ld(audio_features_mod, loudness_shift)
    audio_features_mod = shift_f0(audio_features_mod, pitch_shift)

    af = audio_features if audio_features_mod is None else audio_features_mod

    # Run a batch of predictions.
    start_time = time.time()
    outputs = model(af, training=False)
    audio_gen = model.get_audio_from_outputs(outputs)
    #print(type(audio_gen))
    path = "outputSongs/" + songfileName.split(".")[0] + ".wav"
    try:
      os.makedirs('outputSongs/')
    except:
      print("Not running the os command")  
    #print(path) 
    sf.write(path, np.ravel(audio_gen), DEFAULT_SAMPLE_RATE, subtype='PCM_24')
    print('Prediction took %.1f seconds' % (time.time() - start_time))
    storage.child(uid + "/instrumental/" + songfileName).put(path)

    # Plot
    print('Original')
    play(file_audio)

    print('Resynthesis')
    play(audio_gen)
    

  except Exception as e:
    print(f"{e}")

  return "Success"

# print(storage)
# x = storage.child("audios/Sunflower.mp3").download("Sunflower.mp3")
# print(x)

In [ ]:
cc.run_app(app=app)

Public URL: NgrokTunnel: "https://d90b-34-105-53-70.ngrok.io" -> "http://localhost:8000"


INFO:     Started server process [74]
INFO:uvicorn.error:Started server process [74]
INFO:     Waiting for application startup.
INFO:uvicorn.error:Waiting for application startup.
INFO:     Application startup complete.
INFO:uvicorn.error:Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:uvicorn.error:Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Came
INFO:     103.134.130.206:0 - "GET / HTTP/1.1" 200 OK
INFO:     103.134.130.206:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
Violin
1651857456.213168
Audio features took 12.3 seconds
Hello
Loading dataset statistics from /content/pretrained/dataset_statistics.pkl
ckpt files done
Restoring model took 3.0 seconds
Not running the os command
Prediction took 2.6 seconds
Original


Resynthesis


INFO:     103.134.130.206:0 - "POST /convert HTTP/1.1" 200 OK


In [ ]:
Audio("outputSongs/Sunflower.wav")